In [6]:
# ===============================
# 📊 EMPLOYEE PREDICTION MODEL - 2025 FORECASTING
# Complete Pipeline: Data Processing → Task Time Forecasting → Employee Count Prediction
# Training: 2021-2024 | Prediction Target: 2025
# ===============================

import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import HuberRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🚀 EMPLOYEE PREDICTION MODEL - 2025 FORECASTING PIPELINE")
print("=" * 60)

# ===============================
# 📂 BLOCK 1: DATA LOADING & PREPROCESSING
# ===============================
print("\n" + "=" * 60)
print("📂 BLOCK 1: DATA LOADING & PREPROCESSING")
print("=" * 60)

# Input: Raw staffing data CSV
print("📥 INPUT: staffing_train.csv")

# Load staffing data
staff_df = pd.read_csv("staffing_train.csv")
staff_df['date'] = pd.to_datetime(staff_df['date'])

print(f"📊 LOADED DATA OVERVIEW:")
print(f"   Staffing records: {len(staff_df):,}")
print(f"   Date range: {staff_df['date'].min()} to {staff_df['date'].max()}")
print(f"   Sections: {staff_df['section_id'].nunique()}")

# Generate all dates from 2021-01-01 to 2025-12-31
print("\n🔄 GENERATING COMPLETE DATE RANGE (2021-2025)...")
all_dates = pd.date_range(start="2021-01-01", end="2025-12-31")

# Filter weekends (Saturday=5, Sunday=6)
weekend_dates = all_dates[all_dates.weekday.isin([5, 6])]
weekend_df = pd.DataFrame({'date': weekend_dates})
weekend_df['weekday'] = weekend_df['date'].dt.day_name()

# Sri Lanka holidays for 2021-2025
print("📅 ADDING HOLIDAY DATA...")
web_holiday_data = [
    # 2021 Holidays
    ('2021-01-14', 'Tamil Thai Pongal Day'),
    ('2021-01-28', 'Duruthu Full Moon Poya'),
    ('2021-02-04', 'National Day'),
    ('2021-02-26', 'Navam Full Moon Poya'),
    ('2021-03-11', 'Mahasivarathri Day'),
    ('2021-03-28', 'Madin Full Moon Poya'),
    ('2021-04-02', 'Good Friday'),
    ('2021-04-12', 'Sinhala and Tamil New Year Holiday'),
    ('2021-04-13', 'Sinhala and Tamil New Year Eve'),
    ('2021-04-14', 'Sinhala and Tamil New Year'),
    ('2021-04-26', 'Bak Full Moon Poya'),
    ('2021-05-01', 'Labour Day'),
    ('2021-05-14', 'Id-Ul-Fitr (Ramazan Festival Day)'),
    ('2021-05-24', 'Special Public Holiday'),
    ('2021-05-25', 'Special Public Holiday'),
    ('2021-05-26', 'Vesak Full Moon Poya'),
    ('2021-05-27', 'Vesak Full Moon Poya Holiday'),
    ('2021-06-24', 'Poson Full Moon Poya'),
    ('2021-07-21', 'Idul Adha'),
    ('2021-07-23', 'Esala Full Moon Poya'),
    ('2021-08-22', 'Nikini Full Moon Poya'),
    ('2021-09-20', 'Binara Full Moon Poya'),
    ('2021-10-19', 'Milad-Un-Nabi'),
    ('2021-10-20', 'Vap Full Moon Poya'),
    ('2021-11-04', 'Deepavali Festival Day'),
    ('2021-11-18', 'Ill Full Moon Poya'),
    ('2021-12-18', 'Unduvap Full Moon Poya'),
    ('2021-12-25', 'Christmas Day'),
    # 2022 Holidays
    ('2022-01-14', 'Tamil Thai Pongal Day'),
    ('2022-01-17', 'Duruthu Full Moon Poya'),
    ('2022-02-04', 'National Day'),
    ('2022-02-16', 'Navam Full Moon Poya'),
    ('2022-03-01', 'Mahasivarathri Day'),
    ('2022-03-17', 'Madin Full Moon Poya'),
    ('2022-04-11', 'Special Public Holiday'),
    ('2022-04-12', 'Special Public Holiday'),
    ('2022-04-13', 'Sinhala and Tamil New Year Eve'),
    ('2022-04-14', 'Sinhala and Tamil New Year'),
    ('2022-04-15', 'Good Friday'),
    ('2022-04-16', 'Bak Full Moon Poya'),
    ('2022-05-01', 'Labour Day'),
    ('2022-05-02', 'Labour Day (in lieu)'),
    ('2022-05-03', 'Id-Ul-Fitr'),
    ('2022-05-15', 'Vesak Full Moon Poya'),
    ('2022-05-16', 'Vesak Full Moon Poya Holiday'),
    ('2022-06-13', 'Special Public Holiday'),
    ('2022-06-14', 'Poson Full Moon Poya'),
    ('2022-06-17', 'Sri Lanka Friday Holiday'),
    ('2022-06-24', 'Sri Lanka Friday Holiday'),
    ('2022-07-01', 'Sri Lanka Friday Holiday'),
    ('2022-07-08', 'Sri Lanka Friday Holiday'),
    ('2022-07-10', 'Idul Adha'),
    ('2022-07-13', 'Esala Full Moon Poya'),
    ('2022-07-15', 'Sri Lanka Friday Holiday'),
    ('2022-07-22', 'Sri Lanka Friday Holiday'),
    ('2022-07-29', 'Sri Lanka Friday Holiday'),
    ('2022-08-11', 'Nikini Full Moon Poya'),
    ('2022-09-10', 'Binara Full Moon Poya'),
    ('2022-09-19', 'Special Public Holiday'),
    ('2022-10-09', 'Vap Full Moon Poya'),
    ('2022-10-09', 'Milad-Un-Nabi'),
    ('2022-10-10', 'Milad-Un-Nabi (in lieu)'),
    ('2022-10-24', 'Deepavali Festival Day'),
    ('2022-11-07', 'Ill Full Moon Poya'),
    ('2022-12-07', 'Unduvap Full Moon Poya'),
    ('2022-12-25', 'Christmas Day'),
    ('2022-12-26', 'Special Public Holiday (in lieu)'),
    # 2023 Holidays
    ('2023-01-06', 'Duruthu Full Moon Poya'),
    ('2023-01-15', 'Tamil Thai Pongal Day'),
    ('2023-02-04', 'National Day'),
    ('2023-02-05', 'Navam Full Moon Poya'),
    ('2023-02-18', 'Mahasivarathri Day'),
    ('2023-03-06', 'Madin Full Moon Poya'),
    ('2023-04-05', 'Bak Full Moon Poya'),
    ('2023-04-07', 'Good Friday'),
    ('2023-04-13', 'Sinhala and Tamil New Year Eve'),
    ('2023-04-14', 'Sinhala and Tamil New Year'),
    ('2023-04-22', 'Id-Ul-Fitr'),
    ('2023-05-01', 'Labour Day'),
    ('2023-05-05', 'Vesak Full Moon Poya'),
    ('2023-05-06', 'Vesak Full Moon Poya Holiday'),
    ('2023-06-03', 'Poson Full Moon Poya'),
    ('2023-06-29', 'Idul Adha'),
    ('2023-07-03', 'Esala Full Moon Poya'),
    ('2023-08-01', 'Nikini Full Moon Poya'),
    ('2023-08-30', 'Adhi Nikini Full Moon Poya'),
    ('2023-09-28', 'Milad-Un-Nabi'),
    ('2023-09-29', 'Binara Full Moon Poya'),
    ('2023-10-28', 'Vap Full Moon Poya'),
    ('2023-11-12', 'Deepavali Festival Day'),
    ('2023-11-26', 'Ill Full Moon Poya'),
    ('2023-12-25', 'Christmas Day'),
    ('2023-12-26', 'Unduvap Full Moon Poya'),
    # 2024 Holidays
    ('2024-01-15', 'Tamil Thai Pongal Day'),
    ('2024-01-25', 'Duruthu Full Moon Poya'),
    ('2024-02-04', 'National Day'),
    ('2024-02-23', 'Navam Full Moon Poya'),
    ('2024-03-08', 'Mahasivarathri Day'),
    ('2024-03-24', 'Madin Full Moon Poya'),
    ('2024-03-29', 'Good Friday'),
    ('2024-04-11', 'Id-Ul-Fitr'),
    ('2024-04-12', 'Sinhala and Tamil New Year Eve'),
    ('2024-04-13', 'Sinhala and Tamil New Year'),
    ('2024-04-15', 'Sinhala and Tamil New Year (in lieu)'),
    ('2024-04-23', 'Bak Full Moon Poya'),
    ('2024-05-01', 'Labour Day'),
    ('2024-05-23', 'Vesak Full Moon Poya'),
    ('2024-05-24', 'Vesak Full Moon Poya Holiday'),
    ('2024-06-17', 'Idul Adha'),
    ('2024-06-21', 'Poson Full Moon Poya'),
    ('2024-07-20', 'Esala Full Moon Poya'),
    ('2024-08-19', 'Nikini Full Moon Poya'),
    ('2024-09-16', 'Milad-Un-Nabi'),
    ('2024-09-17', 'Binara Full Moon Poya'),
    ('2024-09-23', 'Public Holiday'),
    ('2024-10-17', 'Vap Full Moon Poya'),
    ('2024-10-31', 'Deepavali Festival Day'),
    ('2024-11-15', 'Ill Full Moon Poya'),
    ('2024-12-14', 'Unduvap Full Moon Poya'),
    ('2024-12-25', 'Christmas Day'),
    # 2025 Holidays
    ('2025-01-13', 'Duruthu Full Moon Poya'),
    ('2025-01-14', 'Tamil Thai Pongal Day'),
    ('2025-02-04', 'National Day'),
    ('2025-02-12', 'Navam Full Moon Poya'),
    ('2025-02-26', 'Mahasivarathri Day'),
    ('2025-03-13', 'Madin Full Moon Poya'),
    ('2025-03-31', 'Id-Ul-Fitr'),
    ('2025-04-12', 'Bak Full Moon Poya'),
    ('2025-04-13', 'Sinhala and Tamil New Year Eve'),
    ('2025-04-14', 'Sinhala and Tamil New Year'),
    ('2025-04-15', 'Special Bank Holiday'),
    ('2025-04-18', 'Good Friday'),
    ('2025-05-01', 'Labour Day'),
    ('2025-05-12', 'Vesak Full Moon Poya'),
    ('2025-05-13', 'Vesak Full Moon Poya Holiday'),
    ('2025-06-07', 'Idul Adha'),
    ('2025-06-10', 'Poson Full Moon Poya'),
    ('2025-07-10', 'Esala Full Moon Poya'),
    ('2025-08-08', 'Nikini Full Moon Poya'),
    ('2025-09-05', 'Milad-Un-Nabi'),
    ('2025-09-07', 'Binara Full Moon Poya'),
    ('2025-10-06', 'Vap Full Moon Poya'),
    ('2025-10-20', 'Deepavali Festival Day'),
    ('2025-11-05', 'Ill Full Moon Poya'),
    ('2025-12-04', 'Unduvap Full Moon Poya'),
    ('2025-12-25', 'Christmas Day'),
]
web_holiday_df = pd.DataFrame(web_holiday_data, columns=['date', 'holiday_name'])
web_holiday_df['date'] = pd.to_datetime(web_holiday_df['date'])

# Exclude specific dates
exclude_dates = [
    '2023-09-29', '2022-05-02', '2023-06-29', '2022-06-24', '2022-07-08', '2024-09-23',
    '2022-06-13', '2022-09-19', '2023-08-30', '2022-07-15', '2022-07-01', '2023-03-06',
    '2021-05-25', '2022-07-22', '2022-07-29', '2021-05-24', '2022-06-17'
]
exclude_dates_dt = pd.to_datetime(exclude_dates)
filtered_holiday = web_holiday_df[~web_holiday_df['date'].isin(exclude_dates_dt)].reset_index(drop=True)

# Create final DataFrame by combining staff, weekend, and filtered_holiday
print("\n🔄 CREATING COMBINED DATASET...")
sections = staff_df['section_id'].unique()
final_rows = []
for date in all_dates:
    for section in sections:
        staff_row = staff_df[(staff_df['date'] == date) & (staff_df['section_id'] == section)]
        employees_on_duty = staff_row['employees_on_duty'].values[0] if not staff_row.empty else 0
        total_task_time_minutes = staff_row['total_task_time_minutes'].values[0] if not staff_row.empty else 0
        weekend_row = weekend_df[weekend_df['date'] == date]
        weekday = weekend_row['weekday'].values[0] if not weekend_row.empty else 0
        holiday_row = filtered_holiday[filtered_holiday['date'] == date]
        holiday_name = holiday_row['holiday_name'].values[0] if not holiday_row.empty else 0
        final_rows.append({
            'date': date,
            'section_id': section,
            'employees_on_duty': employees_on_duty,
            'total_task_time_minutes': total_task_time_minutes,
            'weekday': weekday,
            'holiday_name': holiday_name
        })
final_df = pd.DataFrame(final_rows)
final_df['date'] = pd.to_datetime(final_df['date'])

# Convert 'holiday_name' and 'weekday' to binary columns
final_df['is_holiday'] = final_df['holiday_name'].apply(lambda x: 1 if x != 0 else 0)
final_df['is_weekend'] = final_df['weekday'].apply(lambda x: 1 if x != 0 else 0)

# Drop the original 'holiday_name' and 'weekday' columns
df = final_df.drop(['holiday_name', 'weekday'], axis=1)
print(f"✅ COMPLETE DATASET CREATED: {len(df):,} records")

# Feature engineering
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['dayofweek'] = df['date'].dt.dayofweek
df['quarter'] = df['date'].dt.quarter
df['is_working_day'] = ((df['is_holiday'] == 0) & (df['is_weekend'] == 0)).astype(int)

# Train-Test Split: Now using 2021-2024 for training, 2025 for prediction
train_data = df[df['year'] <= 2024].copy()
prediction_data = df[df['year'] == 2025].copy()

print("📊 FINAL DATA OVERVIEW:")
print(f"   Total records: {len(df):,}")
print(f"   Date range: {df['date'].min()} to {df['date'].max()}")
print(f"   Sections: {df['section_id'].nunique()}")
print(f"   Training data: {len(train_data):,} records (2021-2024)")
print(f"   Prediction target: {len(prediction_data):,} records (2025)")

# ===============================
# 🔧 CONFIGURATION PARAMETERS
# ===============================
print("\n" + "=" * 60)
print("🔧 CONFIGURATION PARAMETERS")
print("=" * 60)

# Section-specific Prophet hyperparameters (tuned with Optuna)
PROPHET_PARAMS = {
    'SEC-001': {
        'changepoint_prior_scale': 0.029254338019126417,
        'seasonality_prior_scale': 0.01030759392216948,
        'holidays_prior_scale': 4.864715869874991,
        'fourier_order_monthly': 10,
        'fourier_order_quarterly': 5
    },
    'SEC-002': {
        'changepoint_prior_scale': 0.22762138954332634,
        'seasonality_prior_scale': 0.014000919327379695,
        'holidays_prior_scale': 0.18467713067993052,
        'fourier_order_monthly': 4,
        'fourier_order_quarterly': 3
    },
    'SEC-003': {
        'changepoint_prior_scale': 0.10098846366877628,
        'seasonality_prior_scale': 1.543871763639119,
        'holidays_prior_scale': 1.21711608065993,
        'fourier_order_monthly': 6,
        'fourier_order_quarterly': 3
    },
    'SEC-004': {
        'changepoint_prior_scale': 0.005151892778933222,
        'seasonality_prior_scale': 0.9459006529861674,
        'holidays_prior_scale': 0.10358719372494084,
        'fourier_order_monthly': 4,
        'fourier_order_quarterly': 3
    },
    'SEC-005': {
        'changepoint_prior_scale': 0.004362487854811211,
        'seasonality_prior_scale': 0.013350235089538814,
        'holidays_prior_scale': 15.35123292088448,
        'fourier_order_monthly': 4,
        'fourier_order_quarterly': 9
    },
    'SEC-006': {
        'changepoint_prior_scale': 0.0015881037419492121,
        'seasonality_prior_scale': 0.03300723715996255,
        'holidays_prior_scale': 14.07408779964586,
        'fourier_order_monthly': 5,
        'fourier_order_quarterly': 4
    }
}

# Default parameters (fallback for any missing sections)
DEFAULT_PROPHET_PARAMS = {
    'changepoint_prior_scale': 0.05,
    'seasonality_prior_scale': 8.0,
    'holidays_prior_scale': 10.0,
    'fourier_order_monthly': 5,
    'fourier_order_quarterly': 3
}

# Optimized HuberRegressor Parameters (from hyperparameter tuning)
OPTIMIZED_HUBER_PARAMS = {
    'SEC-001': {'epsilon': 2.5, 'alpha': 2.3819256646753313e-06, 'max_iter': 450, 'fit_intercept': True},
    'SEC-002': {'epsilon': 1.3, 'alpha': 0.043570676782234655, 'max_iter': 600, 'fit_intercept': True},
    'SEC-003': {'epsilon': 1.3, 'alpha': 5.002881133425486e-06, 'max_iter': 1550, 'fit_intercept': True},
    'SEC-004': {'epsilon': 1.0, 'alpha': 0.002086668812396698, 'max_iter': 300, 'fit_intercept': True},
    'SEC-005': {'epsilon': 1.4, 'alpha': 0.0001800366193826709, 'max_iter': 1350, 'fit_intercept': True},
    'SEC-006': {'epsilon': 2.5, 'alpha': 2.3819256646753313e-06, 'max_iter': 450, 'fit_intercept': True}
}

# Default HuberRegressor parameters (fallback for new sections)
DEFAULT_HUBER_PARAMS = {
    'epsilon': 1.35,
    'alpha': 0.0001,
    'max_iter': 1000,
    'fit_intercept': True
}

# Model parameters
REGRESSION_WEIGHT_THRESHOLD = 0.9  # R² threshold for regression weight
REGRESSION_WEIGHT_HIGH = 0.8       # Weight when R² > threshold
REGRESSION_WEIGHT_LOW = 0.6        # Weight when R² <= threshold

def get_huber_params(section_id):
    """Get optimized HuberRegressor parameters for a section"""
    return OPTIMIZED_HUBER_PARAMS.get(section_id, DEFAULT_HUBER_PARAMS)

print("📋 CONFIGURATION:")
print("   Section-specific Prophet Parameters:")
for section, params in PROPHET_PARAMS.items():
    print(f"   {section}: CPS={params['changepoint_prior_scale']}, SPS={params['seasonality_prior_scale']}, "
          f"HPS={params['holidays_prior_scale']}, Monthly={params['fourier_order_monthly']}, Quarterly={params['fourier_order_quarterly']}")
print(f"   Default fallback parameters available for new sections")

🚀 EMPLOYEE PREDICTION MODEL - 2025 FORECASTING PIPELINE

📂 BLOCK 1: DATA LOADING & PREPROCESSING
📥 INPUT: staffing_train.csv
📊 LOADED DATA OVERVIEW:
   Staffing records: 5,802
   Date range: 2021-01-01 00:00:00 to 2024-12-31 00:00:00
   Sections: 6

🔄 GENERATING COMPLETE DATE RANGE (2021-2025)...
📅 ADDING HOLIDAY DATA...

🔄 CREATING COMBINED DATASET...
📊 LOADED DATA OVERVIEW:
   Staffing records: 5,802
   Date range: 2021-01-01 00:00:00 to 2024-12-31 00:00:00
   Sections: 6

🔄 GENERATING COMPLETE DATE RANGE (2021-2025)...
📅 ADDING HOLIDAY DATA...

🔄 CREATING COMBINED DATASET...
✅ COMPLETE DATASET CREATED: 10,956 records
📊 FINAL DATA OVERVIEW:
   Total records: 10,956
   Date range: 2021-01-01 00:00:00 to 2025-12-31 00:00:00
   Sections: 6
   Training data: 8,766 records (2021-2024)
   Prediction target: 2,190 records (2025)

🔧 CONFIGURATION PARAMETERS
📋 CONFIGURATION:
   Section-specific Prophet Parameters:
   SEC-001: CPS=0.029254338019126417, SPS=0.01030759392216948, HPS=4.8647158698

In [7]:
# ===============================
#  BLOCK 2: TRAINING DATA ANALYSIS (2021-2024)
# ===============================
print("\n" + "=" * 60)
print("📈 BLOCK 2: TRAINING DATA ANALYSIS (2021-2024)")
print("=" * 60)

working_train = train_data[train_data['is_working_day'] == 1]

print("📊 CORRELATION ANALYSIS (Employee vs Task Time):")
correlations = {}
correlation_results = []

for section in working_train['section_id'].unique():
    section_data = working_train[working_train['section_id'] == section]
    if len(section_data) > 20:
        corr = section_data['employees_on_duty'].corr(section_data['total_task_time_minutes'])
        correlations[section] = corr
        correlation_results.append({
            'section_id': section,
            'correlation': corr,
            'data_points': len(section_data)
        })
        print(f"   {section}: {corr:.3f} ({len(section_data)} data points)")

# Create correlation DataFrame
correlation_df = pd.DataFrame(correlation_results)

print(f"\n📈 TRAINING STATISTICS BY SECTION (2021-2024):")
stats = working_train.groupby('section_id').agg({
    'employees_on_duty': ['count', 'mean', 'std', 'min', 'max'],
    'total_task_time_minutes': ['mean', 'std']
}).round(2)

training_stats = []
for section in stats.index:
    emp_stats = stats.loc[section, 'employees_on_duty']
    task_stats = stats.loc[section, 'total_task_time_minutes']
    
    training_stats.append({
        'section_id': section,
        'working_days': emp_stats['count'],
        'emp_mean': emp_stats['mean'],
        'emp_std': emp_stats['std'],
        'emp_min': emp_stats['min'],
        'emp_max': emp_stats['max'],
        'task_mean': task_stats['mean'],
        'task_std': task_stats['std']
    })
    
    print(f"   {section}:")
    print(f"      Working days: {emp_stats['count']:.0f}")
    print(f"      Employees: {emp_stats['mean']:.1f} ± {emp_stats['std']:.1f} (range: {emp_stats['min']:.0f}-{emp_stats['max']:.0f})")
    print(f"      Task time: {task_stats['mean']:.0f} ± {task_stats['std']:.0f} minutes")

# Create training statistics DataFrame
training_stats_df = pd.DataFrame(training_stats)

# ===============================
# 🤖 BLOCK 3: EMPLOYEE REGRESSION MODELS (Updated with 2021-2024)
# ===============================
print("\n" + "=" * 60)
print("🤖 BLOCK 3: EMPLOYEE REGRESSION MODELS (2021-2024 Training)")
print("=" * 60)

employee_models = {}
regression_results = []

print("🔨 BUILDING REGRESSION MODELS (Task Time → Employee Count):")

for section in working_train['section_id'].unique():
    section_data = working_train[working_train['section_id'] == section]

    if len(section_data) > 30:
        # Prepare regression data
        X = section_data[['total_task_time_minutes']].values
        y = section_data['employees_on_duty'].values

        # Get optimized parameters for this section
        huber_params = get_huber_params(section)

        # Build optimized model
        model = HuberRegressor(**huber_params)
        model.fit(X, y)

        # Calculate efficiency metrics
        valid_data = section_data[section_data['employees_on_duty'] > 0].copy()
        valid_data['task_per_emp'] = valid_data['total_task_time_minutes'] / valid_data['employees_on_duty']

        # Store model information
        r2_score_val = model.score(X, y)
        
        employee_models[section] = {
            'regression_model': model,
            'avg_task_per_employee': valid_data['task_per_emp'].median(),
            'min_employees': section_data['employees_on_duty'].min(),
            'max_employees': section_data['employees_on_duty'].max(),
            'training_r2': r2_score_val
        }

        regression_results.append({
            'section_id': section,
            'r2_score': r2_score_val,
            'avg_efficiency': valid_data['task_per_emp'].median(),
            'min_employees': section_data['employees_on_duty'].min(),
            'max_employees': section_data['employees_on_duty'].max(),
            'training_samples': len(section_data)
        })

        print(f"   {section}: R² = {r2_score_val:.3f}, Avg efficiency = {valid_data['task_per_emp'].median():.1f} min/emp")

# Create regression results DataFrame
regression_results_df = pd.DataFrame(regression_results)

print(f"\n🔍 REGRESSION SUMMARY:")
print(regression_results_df.round(2))

# ===============================
# 🔮 BLOCK 4: PROPHET TASK TIME FORECASTING FOR 2025
# ===============================
print("\n" + "=" * 60)
print("🔮 BLOCK 4: PROPHET TASK TIME FORECASTING FOR 2025")
print("=" * 60)

task_forecasts_2025 = {}
task_time_models_2025 = {}
prophet_results_2025 = []

print("📊 FORECASTING 2025 TASK TIME WITH PROPHET (including regressors)...")

for section in working_train['section_id'].unique():
    print(f"   Forecasting {section} for 2025...")
    section_data = working_train[working_train['section_id'] == section].copy()

    if len(section_data) < 50:
        print(f"      ⚠️  Insufficient training data ({len(section_data)} records)")
        continue

    # Prepare Prophet DataFrame
    prophet_df = section_data[['date', 'total_task_time_minutes', 'is_weekend', 'is_holiday']].copy()
    prophet_df.columns = ['ds', 'y', 'is_weekend', 'is_holiday']
    prophet_df = prophet_df.sort_values('ds')

    # Section-specific parameters or fallback
    section_params = PROPHET_PARAMS.get(section, DEFAULT_PROPHET_PARAMS)
    
    # Build Prophet model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoint_prior_scale=section_params['changepoint_prior_scale'],
        seasonality_prior_scale=section_params['seasonality_prior_scale'],
        holidays_prior_scale=section_params['holidays_prior_scale']
    )

    # Add custom seasonalities
    model.add_seasonality(name='monthly', period=30.5, fourier_order=section_params['fourier_order_monthly'])
    model.add_seasonality(name='quarterly', period=91.25, fourier_order=section_params['fourier_order_quarterly'])
    
    # Add regressors
    model.add_regressor('is_weekend', prior_scale=0.5)
    model.add_regressor('is_holiday', prior_scale=1.0)

    # Fit model with regressors
    model.fit(prophet_df[['ds', 'y', 'is_weekend', 'is_holiday']])

    # Prepare 2025 forecast dates with regressors
    prediction_working_2025 = prediction_data[(prediction_data['section_id'] == section) & (prediction_data['is_working_day'] == 1)].copy()
    if len(prediction_working_2025) > 0:
        future_df_2025 = prediction_working_2025[['date', 'is_weekend', 'is_holiday']].copy()
        future_df_2025.columns = ['ds', 'is_weekend', 'is_holiday']
        forecast_2025 = model.predict(future_df_2025)
        forecast_2025['yhat'] = forecast_2025['yhat'].clip(lower=0)

        task_forecasts_2025[section] = {
            'dates': future_df_2025['ds'].values,
            'predictions': forecast_2025['yhat'].values,
            'lower_bound': forecast_2025['yhat_lower'].values,
            'upper_bound': forecast_2025['yhat_upper'].values
        }
        task_time_models_2025[section] = model

        # Training MAE on 2021-2024 data
        train_forecast = model.predict(prophet_df[['ds', 'is_weekend', 'is_holiday']])
        train_mae = mean_absolute_error(prophet_df['y'], train_forecast['yhat'])

        prophet_results_2025.append({
            'section_id': section,
            'training_samples_2021_2024': len(prophet_df),
            'forecast_days_2025': len(future_df_2025),
            'train_mae_2021_2024': train_mae,
            'avg_predicted_task_time_2025': forecast_2025['yhat'].mean(),
            'min_predicted_2025': forecast_2025['yhat'].min(),
            'max_predicted_2025': forecast_2025['yhat'].max(),
            'changepoint_prior_scale': section_params['changepoint_prior_scale'],
            'seasonality_prior_scale': section_params['seasonality_prior_scale'],
            'holidays_prior_scale': section_params['holidays_prior_scale'],
            'fourier_monthly': section_params['fourier_order_monthly'],
            'fourier_quarterly': section_params['fourier_order_quarterly']
        })
        
        print(f"      → Forecasted {len(future_df_2025)} working days in 2025")
        print(f"         Avg: {forecast_2025['yhat'].mean():.1f} min, Range: {forecast_2025['yhat'].min():.1f}-{forecast_2025['yhat'].max():.1f}")
        print(f"         Training MAE (2021-2024): {train_mae:.1f}")

# Create Prophet results DataFrame
prophet_results_2025_df = pd.DataFrame(prophet_results_2025)

print(f"\n PROPHET 2025 FORECAST SUMMARY:")
print(prophet_results_2025_df.round(2))

# ===============================
# 🔄 BLOCK 5: 2025 TASK TIME → EMPLOYEE CONVERSION
# ===============================
print("\n" + "=" * 60)
print("🔄 BLOCK 5: 2025 TASK TIME → EMPLOYEE CONVERSION")
print("=" * 60)

employee_predictions_2025 = {}
conversion_results_2025 = []

print("🔄 CONVERTING 2025 TASK TIME FORECASTS TO EMPLOYEE PREDICTIONS:")

for section, forecast_data in task_forecasts_2025.items():
    if section not in employee_models:
        print(f"   ⚠️  {section}: No regression model available")
        continue

    model_info = employee_models[section]
    dates = forecast_data['dates']
    predicted_task_time = forecast_data['predictions']
    lower_bound = forecast_data['lower_bound']
    upper_bound = forecast_data['upper_bound']

    # Method 1: Regression prediction
    predicted_employees_reg = model_info['regression_model'].predict(predicted_task_time.reshape(-1, 1))
    
    # Method 2: Efficiency-based prediction
    predicted_employees_eff = predicted_task_time / model_info['avg_task_per_employee']

    # Combine predictions based on regression R²
    r2_val = model_info['training_r2']
    weight_reg = REGRESSION_WEIGHT_HIGH if r2_val > REGRESSION_WEIGHT_THRESHOLD else REGRESSION_WEIGHT_LOW

    final_predictions = weight_reg * predicted_employees_reg + (1 - weight_reg) * predicted_employees_eff
    final_predictions = np.round(final_predictions).astype(int)
    
    # Calculate confidence bounds for employees
    employees_lower = np.round(lower_bound / model_info['avg_task_per_employee']).astype(int)
    employees_upper = np.round(upper_bound / model_info['avg_task_per_employee']).astype(int)
    
    # Apply constraints
    final_predictions = np.clip(
        final_predictions,
        max(1, model_info['min_employees']),
        int(model_info['max_employees'] * 1.3)  # Allow 30% above historical max for 2025
    )
    
    employees_lower = np.clip(employees_lower, max(1, model_info['min_employees']), final_predictions)
    employees_upper = np.clip(employees_upper, final_predictions, int(model_info['max_employees'] * 1.5))

    employee_predictions_2025[section] = {
        'dates': dates,
        'predictions': final_predictions,
        'lower_bound': employees_lower,
        'upper_bound': employees_upper
    }
    
    conversion_results_2025.append({
        'section_id': section,
        'forecast_days_2025': len(dates),
        'avg_predicted_employees_2025': final_predictions.mean(),
        'min_predicted_2025': final_predictions.min(),
        'max_predicted_2025': final_predictions.max(),
        'regression_weight': weight_reg,
        'model_r2': r2_val,
        'avg_confidence_range': (employees_upper - employees_lower).mean()
    })

    print(f"   {section}: {len(dates)} working days in 2025")
    print(f"      Avg employees: {final_predictions.mean():.1f} (range: {final_predictions.min()}-{final_predictions.max()})")
    print(f"      Model weight: {weight_reg:.1f}, R²: {r2_val:.3f}")

# Create conversion results DataFrame
conversion_results_2025_df = pd.DataFrame(conversion_results_2025)

print(f"\n 2025 CONVERSION SUMMARY:")
print(conversion_results_2025_df.round(3))

# ===============================
# 📊 BLOCK 6: 2025 COMPLETE PREDICTIONS ASSEMBLY
# ===============================
print("\n" + "=" * 60)
print("📊 BLOCK 6: 2025 COMPLETE PREDICTIONS ASSEMBLY")
print("=" * 60)

# Create complete 2025 predictions DataFrame
complete_final_full_2025_dataset = prediction_data.copy()
complete_final_full_2025_dataset['predicted_employee_count'] = 0
complete_final_full_2025_dataset['predicted_task_time_minutes'] = 0.0
complete_final_full_2025_dataset['employee_lower_bound'] = 0
complete_final_full_2025_dataset['employee_upper_bound'] = 0
complete_final_full_2025_dataset['task_time_lower_bound'] = 0.0
complete_final_full_2025_dataset['task_time_upper_bound'] = 0.0

# Map employee predictions to DataFrame
for section, pred_data in employee_predictions_2025.items():
    for i, date in enumerate(pred_data['dates']):
        mask = (complete_final_full_2025_dataset['section_id'] == section) & (complete_final_full_2025_dataset['date'] == date)
        complete_final_full_2025_dataset.loc[mask, 'predicted_employee_count'] = pred_data['predictions'][i]
        complete_final_full_2025_dataset.loc[mask, 'employee_lower_bound'] = pred_data['lower_bound'][i]
        complete_final_full_2025_dataset.loc[mask, 'employee_upper_bound'] = pred_data['upper_bound'][i]

# Map task time predictions to DataFrame
for section, forecast_data in task_forecasts_2025.items():
    for i, date in enumerate(forecast_data['dates']):
        mask = (complete_final_full_2025_dataset['section_id'] == section) & (complete_final_full_2025_dataset['date'] == date)
        complete_final_full_2025_dataset.loc[mask, 'predicted_task_time_minutes'] = forecast_data['predictions'][i]
        complete_final_full_2025_dataset.loc[mask, 'task_time_lower_bound'] = forecast_data['lower_bound'][i]
        complete_final_full_2025_dataset.loc[mask, 'task_time_upper_bound'] = forecast_data['upper_bound'][i]

print("📊 2025 PREDICTIONS ASSEMBLY COMPLETE:")
print(f"   Total 2025 records: {len(complete_final_full_2025_dataset):,}")
print(f"   Working day predictions: {len(complete_final_full_2025_dataset[complete_final_full_2025_dataset['is_working_day'] == 1]):,}")
print(f"   Sections with predictions: {complete_final_full_2025_dataset[complete_final_full_2025_dataset['predicted_employee_count'] > 0]['section_id'].nunique()}")

print(f"\n🔍 2025 PREDICTIONS SAMPLE:")
working_sample = complete_final_full_2025_dataset[complete_final_full_2025_dataset['is_working_day'] == 1].head(15)
print(working_sample[['date', 'section_id', 'predicted_employee_count', 'predicted_task_time_minutes', 
                      'employee_lower_bound', 'employee_upper_bound']].to_string())

# ===============================
# 📈 BLOCK 7: 2025 PREDICTION ANALYSIS & INSIGHTS
# ===============================
print("\n" + "=" * 60)
print("📈 BLOCK 7: 2025 PREDICTION ANALYSIS & INSIGHTS")
print("=" * 60)

working_2025_predictions = complete_final_full_2025_dataset[complete_final_full_2025_dataset['is_working_day'] == 1].copy()

print(f"📊 2025 WORKFORCE PLANNING INSIGHTS:")
print(f"   Total working days in 2025: {len(working_2025_predictions):,}")

# Section-wise 2025 analysis
section_analysis_2025 = []
print(f"\n📋 SECTION-WISE 2025 PREDICTIONS:")

for section in working_2025_predictions['section_id'].unique():
    section_data = working_2025_predictions[working_2025_predictions['section_id'] == section]
    if len(section_data) > 0:
        
        # Monthly patterns
        section_data['month'] = section_data['date'].dt.month
        monthly_avg = section_data.groupby('month')['predicted_employee_count'].mean()
        
        # Quarterly patterns  
        section_data['quarter'] = section_data['date'].dt.quarter
        quarterly_avg = section_data.groupby('quarter')['predicted_employee_count'].mean()
        
        # Peak and low periods
        peak_month = monthly_avg.idxmax()
        low_month = monthly_avg.idxmin()
        
        section_analysis_2025.append({
            'Section': section,
            'Working_Days_2025': len(section_data),
            'Avg_Employees_2025': section_data['predicted_employee_count'].mean(),
            'Min_Employees_2025': section_data['predicted_employee_count'].min(),
            'Max_Employees_2025': section_data['predicted_employee_count'].max(),
            'Avg_Task_Time_2025': section_data['predicted_task_time_minutes'].mean(),
            'Peak_Month': peak_month,
            'Peak_Month_Avg': monthly_avg[peak_month],
            'Low_Month': low_month,
            'Low_Month_Avg': monthly_avg[low_month],
            'Q1_Avg': quarterly_avg.get(1, 0),
            'Q2_Avg': quarterly_avg.get(2, 0),
            'Q3_Avg': quarterly_avg.get(3, 0),
            'Q4_Avg': quarterly_avg.get(4, 0),
            'Total_Employee_Days_2025': section_data['predicted_employee_count'].sum()
        })

        print(f"   {section}:")
        print(f"      Working days: {len(section_data)}")
        print(f"      Avg employees: {section_data['predicted_employee_count'].mean():.1f} (range: {section_data['predicted_employee_count'].min()}-{section_data['predicted_employee_count'].max()})")
        print(f"      Peak: Month {peak_month} ({monthly_avg[peak_month]:.1f} avg), Low: Month {low_month} ({monthly_avg[low_month]:.1f} avg)")
        print(f"      Quarterly averages: Q1={quarterly_avg.get(1, 0):.1f}, Q2={quarterly_avg.get(2, 0):.1f}, Q3={quarterly_avg.get(3, 0):.1f}, Q4={quarterly_avg.get(4, 0):.1f}")

# Create 2025 analysis DataFrame
section_analysis_2025_df = pd.DataFrame(section_analysis_2025)

# Overall 2025 insights
total_employee_days_2025 = working_2025_predictions['predicted_employee_count'].sum()
avg_daily_workforce_2025 = working_2025_predictions['predicted_employee_count'].mean()
total_task_time_2025 = working_2025_predictions['predicted_task_time_minutes'].sum()

print(f"\n🎯 2025 OVERALL WORKFORCE INSIGHTS:")
print(f"   Total employee-days needed: {total_employee_days_2025:,}")
print(f"   Average daily workforce: {avg_daily_workforce_2025:.1f} employees")
print(f"   Total predicted task time: {total_task_time_2025:,.0f} minutes ({total_task_time_2025/60:,.0f} hours)")
print(f"   Average daily task time: {total_task_time_2025/len(working_2025_predictions):,.0f} minutes per day")

# Monthly workforce planning
print(f"\n📅 2025 MONTHLY WORKFORCE PLANNING:")
monthly_workforce = working_2025_predictions.groupby(working_2025_predictions['date'].dt.month).agg({
    'predicted_employee_count': ['sum', 'mean', 'max'],
    'predicted_task_time_minutes': 'sum'
}).round(1)

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month in range(1, 13):
    if month in monthly_workforce.index:
        emp_sum = monthly_workforce.loc[month, ('predicted_employee_count', 'sum')]
        emp_avg = monthly_workforce.loc[month, ('predicted_employee_count', 'mean')]
        emp_max = monthly_workforce.loc[month, ('predicted_employee_count', 'max')]
        task_sum = monthly_workforce.loc[month, ('predicted_task_time_minutes', 'sum')]
        print(f"   {month_names[month-1]} 2025: {emp_sum:,.0f} employee-days, {emp_avg:.1f} avg daily, {emp_max:.0f} peak daily, {task_sum/60:,.0f} total hours")

# ===============================
# 📈 BLOCK 8: 2025 VISUALIZATIONS
# ===============================
print("\n" + "=" * 60)
print("📈 BLOCK 8: 2025 VISUALIZATIONS")
print("=" * 60)

print("📊 CREATING COMPREHENSIVE 2025 VISUALIZATIONS...")

# Create main 2025 visualization grid
fig = plt.figure(figsize=(20, 16))
plt.suptitle('2025 Employee Workforce Predictions - Comprehensive Analysis', fontsize=16, fontweight='bold')

# 1. Monthly workforce trends
plt.subplot(3, 3, 1)
monthly_totals = working_2025_predictions.groupby(working_2025_predictions['date'].dt.month)['predicted_employee_count'].sum()
plt.bar(range(1, 13), [monthly_totals.get(i, 0) for i in range(1, 13)], color='skyblue', alpha=0.8)
plt.xlabel('Month')
plt.ylabel('Total Employee-Days')
plt.title('2025 Monthly Workforce Requirements')
plt.xticks(range(1, 13), [month_names[i-1] for i in range(1, 13)], rotation=45)
plt.grid(True, alpha=0.3)

# 2. Section-wise average employees
plt.subplot(3, 3, 2)
plt.bar(section_analysis_2025_df['Section'], section_analysis_2025_df['Avg_Employees_2025'], color='lightcoral', alpha=0.8)
plt.xlabel('Section')
plt.ylabel('Average Employees')
plt.title('2025 Average Employees by Section')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 3. Peak vs Low month comparison
plt.subplot(3, 3, 3)
x = np.arange(len(section_analysis_2025_df))
width = 0.35
plt.bar(x - width/2, section_analysis_2025_df['Peak_Month_Avg'], width, label='Peak Month', alpha=0.8, color='red')
plt.bar(x + width/2, section_analysis_2025_df['Low_Month_Avg'], width, label='Low Month', alpha=0.8, color='green')
plt.xlabel('Section')
plt.ylabel('Average Employees')
plt.title('2025 Peak vs Low Month Staffing')
plt.xticks(x, section_analysis_2025_df['Section'], rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# 4. Quarterly workforce distribution
plt.subplot(3, 3, 4)
quarterly_data = section_analysis_2025_df[['Q1_Avg', 'Q2_Avg', 'Q3_Avg', 'Q4_Avg']].mean()
plt.pie(quarterly_data, labels=['Q1', 'Q2', 'Q3', 'Q4'], autopct='%1.1f%%', startangle=90)
plt.title('2025 Quarterly Workforce Distribution')

# 5. Task time vs Employee count correlation
plt.subplot(3, 3, 5)
plt.scatter(working_2025_predictions['predicted_task_time_minutes'], 
           working_2025_predictions['predicted_employee_count'], alpha=0.6, s=20)
plt.xlabel('Predicted Task Time (minutes)')
plt.ylabel('Predicted Employee Count')
plt.title('2025 Task Time vs Employee Relationship')
plt.grid(True, alpha=0.3)

# 6. Working days distribution by section
plt.subplot(3, 3, 6)
plt.bar(section_analysis_2025_df['Section'], section_analysis_2025_df['Working_Days_2025'], color='orange', alpha=0.8)
plt.xlabel('Section')
plt.ylabel('Working Days')
plt.title('2025 Working Days by Section')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 7-9. Section-specific time series for top 3 sections by workforce
top_sections = section_analysis_2025_df.nlargest(3, 'Total_Employee_Days_2025')['Section'].tolist()
for i, section in enumerate(top_sections, 7):
    plt.subplot(3, 3, i)
    section_data = working_2025_predictions[working_2025_predictions['section_id'] == section]
    # Resample to weekly averages for cleaner visualization
    section_weekly = section_data.set_index('date').resample('W')['predicted_employee_count'].mean()
    plt.plot(section_weekly.index, section_weekly.values, linewidth=2, marker='o', markersize=3)
    plt.xlabel('2025 Date')
    plt.ylabel('Avg Weekly Employees')
    plt.title(f'{section} - 2025 Weekly Trends')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('2025_workforce_predictions_comprehensive.png', dpi=300, bbox_inches='tight')
plt.close()

# Create confidence intervals visualization
print("📊 CREATING 2025 CONFIDENCE INTERVALS VISUALIZATION...")
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('2025 Employee Predictions with Confidence Intervals by Section', fontsize=14, fontweight='bold')

sections_with_predictions = working_2025_predictions[working_2025_predictions['predicted_employee_count'] > 0]['section_id'].unique()
for i, section in enumerate(sections_with_predictions[:6]):  # Top 6 sections
    row = i // 3
    col = i % 3
    ax = axes[row, col]
    
    section_data = working_2025_predictions[working_2025_predictions['section_id'] == section].copy()
    section_data = section_data.sort_values('date')
    
    # Monthly aggregation for cleaner visualization
    monthly_data = section_data.set_index('date').resample('M').agg({
        'predicted_employee_count': 'mean',
        'employee_lower_bound': 'mean',
        'employee_upper_bound': 'mean'
    })
    
    ax.plot(monthly_data.index, monthly_data['predicted_employee_count'], 'b-', linewidth=2, label='Predicted')
    ax.fill_between(monthly_data.index, monthly_data['employee_lower_bound'], 
                   monthly_data['employee_upper_bound'], alpha=0.3, color='blue', label='Confidence Range')
    ax.set_title(f'{section} - 2025 Monthly Predictions')
    ax.set_xlabel('2025 Month')
    ax.set_ylabel('Employees')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('2025_confidence_intervals_by_section.png', dpi=300, bbox_inches='tight')
plt.close()

# ===============================
# 💾 BLOCK 9: SAVE 2025 RESULTS
# ===============================
print("\n" + "=" * 60)
print("💾 BLOCK 9: SAVE 2025 RESULTS")
print("=" * 60)

# Save all important 2025 DataFrames
complete_final_full_2025_dataset.to_csv('complete_final_full_2025_dataset.csv', index=False)
section_analysis_2025_df.to_csv('2025_workforce_analysis_by_section.csv', index=False)
working_2025_predictions.to_csv('2025_working_days_predictions.csv', index=False)
correlation_df.to_csv('2021_2024_training_correlations.csv', index=False)
training_stats_df.to_csv('2021_2024_training_statistics.csv', index=False)
regression_results_df.to_csv('2021_2024_regression_model_results.csv', index=False)
prophet_results_2025_df.to_csv('2025_prophet_forecast_results.csv', index=False)
conversion_results_2025_df.to_csv('2025_task_to_employee_conversion.csv', index=False)

# Save monthly workforce planning data
monthly_workforce_df = pd.DataFrame({
    'Month': range(1, 13),
    'Month_Name': month_names,
    'Total_Employee_Days': [monthly_workforce.loc[i, ('predicted_employee_count', 'sum')] if i in monthly_workforce.index else 0 for i in range(1, 13)],
    'Avg_Daily_Employees': [monthly_workforce.loc[i, ('predicted_employee_count', 'mean')] if i in monthly_workforce.index else 0 for i in range(1, 13)],
    'Peak_Daily_Employees': [monthly_workforce.loc[i, ('predicted_employee_count', 'max')] if i in monthly_workforce.index else 0 for i in range(1, 13)],
    'Total_Task_Hours': [monthly_workforce.loc[i, ('predicted_task_time_minutes', 'sum')]/60 if i in monthly_workforce.index else 0 for i in range(1, 13)]
})
monthly_workforce_df.to_csv('2025_monthly_workforce_planning.csv', index=False)

print("✅ 2025 FORECASTING COMPLETE!")
print(f"\n📁 FILES SAVED:")
print(f"   - complete_final_full_2025_dataset.csv: ⭐ MAIN OUTPUT - Complete 2025 predictions with confidence intervals")
print(f"   - 2025_workforce_analysis_by_section.csv: Section-wise 2025 workforce analysis")
print(f"   - 2025_working_days_predictions.csv: Working days only 2025 predictions")
print(f"   - 2025_monthly_workforce_planning.csv: Monthly workforce planning summary")
print(f"   - 2021_2024_training_correlations.csv: Updated training correlations")
print(f"   - 2021_2024_training_statistics.csv: Updated training statistics")
print(f"   - 2021_2024_regression_model_results.csv: Updated regression model performance")
print(f"   - 2025_prophet_forecast_results.csv: 2025 Prophet forecasting results")
print(f"   - 2025_task_to_employee_conversion.csv: 2025 conversion metrics")

print("\n📊 FINAL 2025 WORKFORCE SUMMARY:")
print(section_analysis_2025_df.round(2))

print(f"\n🎯 2025 KEY METRICS:")
print(f"   📅 Total working days: {len(working_2025_predictions):,}")
print(f"   👥 Total employee-days needed: {total_employee_days_2025:,}")
print(f"   📈 Average daily workforce: {avg_daily_workforce_2025:.1f} employees")
print(f"   ⏱️  Total predicted task time: {total_task_time_2025/60:,.0f} hours")
print(f"   🏢 Sections covered: {len(section_analysis_2025_df)}")

# Workforce planning recommendations
print(f"\n💡 2025 WORKFORCE PLANNING RECOMMENDATIONS:")
peak_month = monthly_workforce[('predicted_employee_count', 'sum')].idxmax()
low_month = monthly_workforce[('predicted_employee_count', 'sum')].idxmin()
print(f"   📈 Peak staffing month: {month_names[peak_month-1]} ({monthly_workforce.loc[peak_month, ('predicted_employee_count', 'sum')]:,.0f} employee-days)")
print(f"   📉 Lowest staffing month: {month_names[low_month-1]} ({monthly_workforce.loc[low_month, ('predicted_employee_count', 'sum')]:,.0f} employee-days)")

# Identify sections needing most attention
high_variance_sections = section_analysis_2025_df[section_analysis_2025_df['Max_Employees_2025'] - section_analysis_2025_df['Min_Employees_2025'] > 3]
if len(high_variance_sections) > 0:
    print(f"   ⚠️  Sections with high staffing variability:")
    for _, section in high_variance_sections.iterrows():
        print(f"      {section['Section']}: {section['Min_Employees_2025']}-{section['Max_Employees_2025']} employees (range: {section['Max_Employees_2025']-section['Min_Employees_2025']})")

print(f"\n🎉 2025 EMPLOYEE FORECASTING PIPELINE COMPLETED SUCCESSFULLY!")
print(f"📋 Main output file: 'complete_final_full_2025_dataset.csv'")
print("=" * 60)


📈 BLOCK 2: TRAINING DATA ANALYSIS (2021-2024)
📊 CORRELATION ANALYSIS (Employee vs Task Time):
   SEC-001: 0.890 (968 data points)
   SEC-002: 0.911 (968 data points)
   SEC-003: 0.868 (968 data points)
   SEC-004: 0.897 (968 data points)
   SEC-005: 0.982 (968 data points)
   SEC-006: 0.964 (968 data points)

📈 TRAINING STATISTICS BY SECTION (2021-2024):
   SEC-001:
      Working days: 968
      Employees: 2.9 ± 1.5 (range: 0-12)
      Task time: 988 ± 572 minutes
   SEC-002:
      Working days: 968
      Employees: 4.3 ± 1.7 (range: 0-12)
      Task time: 1576 ± 656 minutes
   SEC-003:
      Working days: 968
      Employees: 2.9 ± 1.4 (range: 0-9)
      Task time: 1019 ± 506 minutes
   SEC-004:
      Working days: 968
      Employees: 3.1 ± 1.5 (range: 0-11)
      Task time: 1089 ± 580 minutes
   SEC-005:
      Working days: 968
      Employees: 7.2 ± 3.6 (range: 0-26)
      Task time: 2846 ± 1498 minutes
   SEC-006:
      Working days: 968
      Employees: 6.3 ± 2.6 (range: 0-19)
 

14:46:08 - cmdstanpy - INFO - Chain [1] start processing
14:46:08 - cmdstanpy - INFO - Chain [1] done processing
14:46:08 - cmdstanpy - INFO - Chain [1] done processing
14:46:09 - cmdstanpy - INFO - Chain [1] start processing
14:46:09 - cmdstanpy - INFO - Chain [1] start processing


      → Forecasted 239 working days in 2025
         Avg: 856.3 min, Range: 623.0-1105.2
         Training MAE (2021-2024): 413.9
   Forecasting SEC-002 for 2025...


14:46:09 - cmdstanpy - INFO - Chain [1] done processing


      → Forecasted 239 working days in 2025
         Avg: 1395.8 min, Range: 1149.4-1664.4
         Training MAE (2021-2024): 495.6
   Forecasting SEC-003 for 2025...


...

      → Forecasted 239 working days in 2025
         Avg: 915.9 min, Range: 715.7-1101.5
         Training MAE (2021-2024): 428.3
   Forecasting SEC-005 for 2025...


14:46:11 - cmdstanpy - INFO - Chain [1] start processing
14:46:11 - cmdstanpy - INFO - Chain [1] done processing
14:46:11 - cmdstanpy - INFO - Chain [1] done processing
14:46:11 - cmdstanpy - INFO - Chain [1] start processing
14:46:11 - cmdstanpy - INFO - Chain [1] start processing


      → Forecasted 239 working days in 2025
         Avg: 2585.2 min, Range: 1707.6-3380.7
         Training MAE (2021-2024): 1104.4
   Forecasting SEC-006 for 2025...


14:46:11 - cmdstanpy - INFO - Chain [1] done processing


      → Forecasted 239 working days in 2025
         Avg: 1942.7 min, Range: 1645.7-2341.7
         Training MAE (2021-2024): 804.4

 PROPHET 2025 FORECAST SUMMARY:
  section_id  training_samples_2021_2024  forecast_days_2025  \
0    SEC-001                         968                 239   
1    SEC-002                         968                 239   
2    SEC-003                         968                 239   
3    SEC-004                         968                 239   
4    SEC-005                         968                 239   
5    SEC-006                         968                 239   

   train_mae_2021_2024  avg_predicted_task_time_2025  min_predicted_2025  \
0               413.92                        856.35              622.99   
1               495.59                       1395.76             1149.42   
2               379.07                        852.98              607.37   
3               428.30                        915.88              715.66   
4     

In [8]:
task_2_evaluation_input=pd.read_csv("task2_test_inputs.csv")

In [9]:
print(task_2_evaluation_input.head())

                                     row_id        date section_id
0  f944201f75875b49278c1328d019123f5e7d6f67  2025-01-01    SEC-001
1  3f7b4944e3128a8dc35c448db69452b7e0e7fb6d  2025-01-01    SEC-002
2  d1d8c20b85c9d7dc6279811e1cbc411230d5c867  2025-01-01    SEC-003
3  61c05b152a5ddea1005a630cf4f062d68f9cb333  2025-01-01    SEC-004
4  a40f2ccf6a5a8a68c9e02bfaf89045c56db3e5ae  2025-01-01    SEC-005


In [15]:
print(task_2_evaluation_input.tail())
print("\nColumns in task_2_evaluation_input:")
print(task_2_evaluation_input.columns)

                                        row_id       date section_id  year  \
1435  39362157fdd43fe05210ac44b742fb126ebd782e 2025-12-31    SEC-002  2025   
1436  1e2eab2149c41956ee0ceb238fa127c15a3fe881 2025-12-31    SEC-003  2025   
1437  2913839a5b1dc8ac4e90d025b4208ccca35d45b6 2025-12-31    SEC-004  2025   
1438  9c87a3f79eddcbcdcd0fd10a9bd2efea6d49e078 2025-12-31    SEC-005  2025   
1439  ce36ffec89c04bd1c6f8b55a3d68e3a5f2870687 2025-12-31    SEC-006  2025   

      month  dayofweek  quarter  is_weekend  is_holiday  is_working_day  
1435     12          2        4           0           0               1  
1436     12          2        4           0           0               1  
1437     12          2        4           0           0               1  
1438     12          2        4           0           0               1  
1439     12          2        4           0           0               1  

Columns in task_2_evaluation_input:
Index(['row_id', 'date', 'section_id', 'year', 'mo

In [11]:
# ===============================
# 🥒 MODEL EXPORT TO PICKLE FILES
# ===============================
print("\n" + "=" * 60)
print("🥒 EXPORTING MODELS TO PICKLE FILES")
print("=" * 60)

import pickle

# Create a single dictionary with all model components
all_models = {
    'employee_models': employee_models,
    'task_time_models_2025': task_time_models_2025,
    'PROPHET_PARAMS': PROPHET_PARAMS,
    'DEFAULT_PROPHET_PARAMS': DEFAULT_PROPHET_PARAMS,
    'OPTIMIZED_HUBER_PARAMS': OPTIMIZED_HUBER_PARAMS,
    'DEFAULT_HUBER_PARAMS': DEFAULT_HUBER_PARAMS,
    'REGRESSION_WEIGHT_THRESHOLD': REGRESSION_WEIGHT_THRESHOLD,
    'REGRESSION_WEIGHT_HIGH': REGRESSION_WEIGHT_HIGH,
    'REGRESSION_WEIGHT_LOW': REGRESSION_WEIGHT_LOW
}

# Export all models and parameters to a single pickle file
with open('predictions_full.pkl', 'wb') as f:
    pickle.dump(all_models, f)
print("✅ Exported all models and parameters to predictions_full.pkl")


🥒 EXPORTING MODELS TO PICKLE FILES
✅ Exported all models and parameters to predictions_full.pkl


In [12]:
# ===============================
# 📤 LOAD PICKLE MODEL FILES FOR PREDICTION
# ===============================
print("\n" + "=" * 60)
print("📤 LOADING MODELS FROM PICKLE FILE")
print("=" * 60)

import pickle

# Load all models and parameters from the single pickle file
try:
    with open('predictions_full.pkl', 'rb') as f:
        all_models = pickle.load(f)
    print("✅ Successfully loaded all models and parameters")
    
    # Extract individual components from the loaded dictionary
    loaded_employee_models = all_models['employee_models']
    loaded_task_time_models = all_models['task_time_models_2025']
    PROPHET_PARAMS = all_models['PROPHET_PARAMS']
    DEFAULT_PROPHET_PARAMS = all_models['DEFAULT_PROPHET_PARAMS']
    OPTIMIZED_HUBER_PARAMS = all_models['OPTIMIZED_HUBER_PARAMS']
    DEFAULT_HUBER_PARAMS = all_models['DEFAULT_HUBER_PARAMS']
    REGRESSION_WEIGHT_THRESHOLD = all_models['REGRESSION_WEIGHT_THRESHOLD']
    REGRESSION_WEIGHT_HIGH = all_models['REGRESSION_WEIGHT_HIGH']
    REGRESSION_WEIGHT_LOW = all_models['REGRESSION_WEIGHT_LOW']
    
    print("📊 Loaded model components:")
    print(f"   Employee models for {len(loaded_employee_models)} sections")
    print(f"   Task time models for {len(loaded_task_time_models)} sections")
    
except FileNotFoundError:
    print("⚠️ Model file not found. Please run the model export cell first.")
    # Use the ones in memory as fallback
    loaded_employee_models = employee_models
    loaded_task_time_models = task_time_models_2025
    print("⚠️ Using models from current memory instead")


📤 LOADING MODELS FROM PICKLE FILE
✅ Successfully loaded all models and parameters
📊 Loaded model components:
   Employee models for 6 sections
   Task time models for 6 sections
✅ Successfully loaded all models and parameters
📊 Loaded model components:
   Employee models for 6 sections
   Task time models for 6 sections


In [ ]:
# ===============================
# 🔍 PREPARE EVALUATION INPUT DATA
# ===============================
print("\n" + "=" * 60)
print("🔍 PREPARING EVALUATION INPUT DATA")
print("=" * 60)

# Display information about the evaluation input
print(f"📊 Evaluation Input Overview:")
print(f"   Total rows: {len(task_2_evaluation_input):,}")
print(f"   Columns: {', '.join(task_2_evaluation_input.columns)}")

# Convert date to datetime if needed
if 'date' in task_2_evaluation_input.columns and not pd.api.types.is_datetime64_any_dtype(task_2_evaluation_input['date']):
    task_2_evaluation_input['date'] = pd.to_datetime(task_2_evaluation_input['date'])
    print("✅ Converted date column to datetime")

# Add necessary features similar to the training data
task_2_evaluation_input['year'] = task_2_evaluation_input['date'].dt.year
task_2_evaluation_input['month'] = task_2_evaluation_input['date'].dt.month
task_2_evaluation_input['dayofweek'] = task_2_evaluation_input['date'].dt.dayofweek
task_2_evaluation_input['quarter'] = task_2_evaluation_input['date'].dt.quarter

# Check if we need to add weekend/holiday features
if 'is_weekend' not in task_2_evaluation_input.columns:
    # Check if the dates are in the weekend (5=Saturday, 6=Sunday)
    task_2_evaluation_input['is_weekend'] = task_2_evaluation_input['date'].dt.dayofweek.isin([5, 6]).astype(int)
    print("✅ Added weekend flag")
    
if 'is_holiday' not in task_2_evaluation_input.columns:
    # Add holiday information (reuse the holiday data from before)
    filtered_holiday_dict = dict(zip(pd.to_datetime(filtered_holiday['date']), [1] * len(filtered_holiday)))
    task_2_evaluation_input['is_holiday'] = task_2_evaluation_input['date'].map(
        lambda x: 1 if x in filtered_holiday_dict else 0)
    print("✅ Added holiday flag")
    
if 'is_working_day' not in task_2_evaluation_input.columns:
    # Add working day flag (not weekend and not holiday)
    task_2_evaluation_input['is_working_day'] = ((task_2_evaluation_input['is_holiday'] == 0) & 
                                                (task_2_evaluation_input['is_weekend'] == 0)).astype(int)
    print("✅ Added working day flag")

# Print summary of working vs non-working days
working_count = task_2_evaluation_input['is_working_day'].sum()
weekend_count = task_2_evaluation_input['is_weekend'].sum()
holiday_count = (task_2_evaluation_input['is_holiday'] & ~task_2_evaluation_input['is_weekend']).sum()

print(f"\n📅 Day Type Summary:")
print(f"   Working Days: {working_count}")
print(f"   Weekend Days: {weekend_count}")
print(f"   Holiday Days (excluding weekends): {holiday_count}")
print(f"   Total: {len(task_2_evaluation_input)}")

# Show sample of prepared data
print("\n📋 Sample of prepared evaluation data:")
print(task_2_evaluation_input.head())


🔍 PREPARING EVALUATION INPUT DATA
📊 Evaluation Input Overview:
   Total rows: 1,440
   Columns: row_id, date, section_id
✅ Converted date column to datetime
✅ Added weekend, holiday, and working day features

📋 Sample of prepared evaluation data:
                                     row_id       date section_id  year  \
0  f944201f75875b49278c1328d019123f5e7d6f67 2025-01-01    SEC-001  2025   
1  3f7b4944e3128a8dc35c448db69452b7e0e7fb6d 2025-01-01    SEC-002  2025   
2  d1d8c20b85c9d7dc6279811e1cbc411230d5c867 2025-01-01    SEC-003  2025   
3  61c05b152a5ddea1005a630cf4f062d68f9cb333 2025-01-01    SEC-004  2025   
4  a40f2ccf6a5a8a68c9e02bfaf89045c56db3e5ae 2025-01-01    SEC-005  2025   

   month  dayofweek  quarter  is_weekend  is_holiday  is_working_day  
0      1          2        1           0           0               1  
1      1          2        1           0           0               1  
2      1          2        1           0           0               1  
3      1         

In [ ]:
# ===============================
# 🔮 MAKE PREDICTIONS ON EVALUATION DATA
# ===============================
print("\n" + "=" * 60)
print("🔮 MAKING PREDICTIONS ON EVALUATION DATA")
print("=" * 60)

# Check what columns are available
print(f"Columns in evaluation data: {task_2_evaluation_input.columns.tolist()}")

# Create a DataFrame to store the results
predictions_df = task_2_evaluation_input[['row_id']].copy()
predictions_df['true_required_employees'] = 0

# Make predictions for each row
print(f"🔍 Making predictions for {len(task_2_evaluation_input)} rows...")

# First, predict task times for each section
section_task_times = {}
sections_to_process = task_2_evaluation_input['section_id'].unique()

print("Step 1: Predicting task times using Prophet models...")
for section in sections_to_process:
    if section in loaded_task_time_models:
        # Get the Prophet model for this section
        prophet_model = loaded_task_time_models[section]
        
        # Prepare future dataframe for this section
        section_dates = task_2_evaluation_input[task_2_evaluation_input['section_id'] == section]
        future_df = section_dates[['date', 'is_weekend', 'is_holiday']].copy()
        future_df.columns = ['ds', 'is_weekend', 'is_holiday']
        
        # Make predictions
        forecast = prophet_model.predict(future_df)
        
        # Store predictions in a dictionary keyed by (section, date) for quick lookup
        for i, row in forecast.iterrows():
            date = row['ds']
            predicted_time = max(0, row['yhat'])  # Ensure non-negative
            section_task_times[(section, date)] = predicted_time
        
        print(f"  - {section}: Predicted task times for {len(future_df)} dates")
    else:
        print(f"  - ⚠️ No Prophet model available for {section}, will use averages")
        
        # Use average from training data as fallback
        section_data = working_train[working_train['section_id'] == section]
        if len(section_data) > 0:
            avg_task_time = section_data['total_task_time_minutes'].mean()
        else:
            avg_task_time = working_train['total_task_time_minutes'].mean()
            
        # Apply to all dates for this section
        section_dates = task_2_evaluation_input[task_2_evaluation_input['section_id'] == section]['date'].unique()
        for date in section_dates:
            section_task_times[(section, date)] = avg_task_time

print("\nStep 2: Converting predicted task times to employee counts...")
for index, row in task_2_evaluation_input.iterrows():
    section = row['section_id']
    date = row['date']
    is_weekend = row['is_weekend']
    is_holiday = row['is_holiday']
    row_id = row['row_id']
    
    # Check if it's a weekend or holiday
    if is_weekend == 1 or is_holiday == 1:
        # For weekends or holidays, set employee count to 0
        predictions_df.loc[predictions_df['row_id'] == row_id, 'true_required_employees'] = 0
        continue
    
    # For working days (not weekends or holidays), proceed with prediction
    # Get the predicted task time for this section and date
    task_time = section_task_times.get((section, date), 0)
    
    # Check if we have models for this section
    if section in loaded_employee_models:
        model_info = loaded_employee_models[section]
        
        # Method 1: Use regression model to predict
        predicted_employees_reg = model_info['regression_model'].predict(
            np.array([[task_time]])
        )[0]
        
        # Method 2: Use efficiency-based calculation
        predicted_employees_eff = task_time / model_info['avg_task_per_employee'] if model_info['avg_task_per_employee'] > 0 else 0
        
        # Combine predictions based on regression R²
        r2_val = model_info['training_r2']
        weight_reg = REGRESSION_WEIGHT_HIGH if r2_val > REGRESSION_WEIGHT_THRESHOLD else REGRESSION_WEIGHT_LOW
        
        final_prediction = weight_reg * predicted_employees_reg + (1 - weight_reg) * predicted_employees_eff
        
        # For working days, ensure at least 1 employee
        final_prediction = max(1, min(int(round(final_prediction)), int(model_info['max_employees'] * 1.3)))
        
        predictions_df.loc[predictions_df['row_id'] == row_id, 'true_required_employees'] = final_prediction
    else:
        # Fallback for sections without models - use a simple formula based on average efficiency
        avg_efficiency = 60  # Assuming 60 minutes per employee is a reasonable default
        predicted_employees = task_time / avg_efficiency if avg_efficiency > 0 else 1
        
        # For working days, ensure at least 1 employee
        final_prediction = max(1, int(round(predicted_employees)))
        
        predictions_df.loc[predictions_df['row_id'] == row_id, 'true_required_employees'] = final_prediction

print(f"✅ Predictions complete for {len(predictions_df)} rows")
print("\n📊 Predictions Summary:")
print(f"   Min predicted employees: {predictions_df['true_required_employees'].min()}")
print(f"   Max predicted employees: {predictions_df['true_required_employees'].max()}")
print(f"   Mean predicted employees: {predictions_df['true_required_employees'].mean():.2f}")
print("\n📋 Sample predictions:")
print(predictions_df.head(10))

# Count records by working day status
working_days = len(predictions_df[predictions_df['true_required_employees'] > 0])
non_working_days = len(predictions_df[predictions_df['true_required_employees'] == 0])
print(f"\n📅 Working Days: {working_days} (with employees > 0)")
print(f"📅 Non-Working Days: {non_working_days} (with employees = 0)")


🔮 MAKING PREDICTIONS ON EVALUATION DATA
Columns in evaluation data: ['row_id', 'date', 'section_id', 'year', 'month', 'dayofweek', 'quarter', 'is_weekend', 'is_holiday', 'is_working_day']
🔍 Making predictions for 1440 rows...
Step 1: Predicting task times using Prophet models...
  - SEC-001: Predicted task times for 240 dates
  - SEC-002: Predicted task times for 240 dates
  - SEC-003: Predicted task times for 240 dates
  - SEC-001: Predicted task times for 240 dates
  - SEC-002: Predicted task times for 240 dates
  - SEC-003: Predicted task times for 240 dates
  - SEC-004: Predicted task times for 240 dates
  - SEC-005: Predicted task times for 240 dates
  - SEC-006: Predicted task times for 240 dates

Step 2: Converting predicted task times to employee counts...
  - SEC-004: Predicted task times for 240 dates
  - SEC-005: Predicted task times for 240 dates
  - SEC-006: Predicted task times for 240 dates

Step 2: Converting predicted task times to employee counts...
✅ Predictions com

In [17]:
# ===============================
# 💾 SAVE PREDICTIONS TO CSV FILE
# ===============================
print("\n" + "=" * 60)
print("💾 SAVING PREDICTIONS TO CSV FILE")
print("=" * 60)

# Define the output filename
output_filename = 'task2_predictions.csv'

# Save only the required columns: row_id and true_required_employees
predictions_output = predictions_df[['row_id', 'true_required_employees']]
predictions_output.to_csv(output_filename, index=False)

print(f"✅ Successfully saved predictions to {output_filename}")
print(f"   Total rows: {len(predictions_output):,}")
print(f"   Columns: {', '.join(predictions_output.columns)}")

# Display the first few rows of the saved file
print("\n📋 Preview of saved predictions:")
print(predictions_output.head(10))

# Display output statistics by section
print("\n📊 Prediction Statistics by Section:")
merged_df = task_2_evaluation_input.merge(predictions_df, on='row_id')
section_stats = merged_df.groupby('section_id')['true_required_employees'].agg(['count', 'mean', 'min', 'max'])
print(section_stats)


💾 SAVING PREDICTIONS TO CSV FILE
✅ Successfully saved predictions to task2_predictions.csv
   Total rows: 1,440
   Columns: row_id, true_required_employees

📋 Preview of saved predictions:
                                     row_id  true_required_employees
0  f944201f75875b49278c1328d019123f5e7d6f67                        2
1  3f7b4944e3128a8dc35c448db69452b7e0e7fb6d                        4
2  d1d8c20b85c9d7dc6279811e1cbc411230d5c867                        2
3  61c05b152a5ddea1005a630cf4f062d68f9cb333                        3
4  a40f2ccf6a5a8a68c9e02bfaf89045c56db3e5ae                        6
5  23738dca3dcfaa9fa43905c4ee40849d25154604                        5
6  cf95a3d507560c0c1c12b4362f994480f3af8eb1                        2
7  99332e7824cd87585e2edc022daa70ab1ba0287a                        4
8  ff8e449af42069a8e80e924daef37b2f4c1588ae                        2
9  f7b15dc612ea5c45bbd82b175868196e3d4a49c7                        3

📊 Prediction Statistics by Section:
            co